<div class = "alert alert-block alert-success">
    
# <span style='color:Blue'> NOTEBOOK 0: Welcome to Nightingale's model training overview! </span>


## This notebook will give an overview of Nightingale's model training process and explain some crucial set-up steps prior to starting data pre-processing. There is no code to run in this notebook, but you will need to locate and edit two files. 

# <span style='color:Blue'> Nightingale organizes model training into the following steps and notebooks:

> ## <span style='color:Blue'> STEP 0.1 </span> - Modify the training *configuration* file to match your dataset and environment (steps detailed in *this* notebook)
> ## <span style='color:Blue'> STEP 0.2 </span> - Edit the *category label dictionary* (in Nightingale/ Training/ libs/ label_name_dict) to include your dataset's categories and training-NMS threshold settings for each category (steps detailed in *this* notebook)
> ## <span style='color:Blue'> STEP 1 </span> - Cleaning and formatting the data for Nightingale (see Notebook-1)
> ## <span style='color:Blue'> STEP 2 </span> - Converting NITF imagery and formatted groundtruth CSV's to a Tensorflow Record for training (see Notebook-2)
> ## <span style='color:Blue'> STEP 3 </span> - Training the model (see Notebook-3)
> ## <span style='color:Blue'> STEP 4 </span> - Exporting the model for running inference (see Notebook-4)

<div class = "alert alert-block alert-success">
    
# <span style='color:Blue'> Complete steps 0.1 & 0.2 </span>

# <span style='color:Blue'> Step 0.1 - Edit the configuration file to match your dataset and environment
    
* ## Using a text editor of your choice, open *cfgs.py* in Nightingale/Training/libs/configs/

* ## Modify the *Basic Settings* code block at the top of the config file to reflect your dataset, network choice, available GPUs, and path to the Training directory etc. See the code block comments for explanation of each setting. 

* ## If you don't want to edit these yet, that's ok, but you will need the DATASET_NAME set before running Notebook #2 and have all the settings finalized before training the network (Notebook #3)

# <span style='color:green'> The basic settings code block in the config file looks like the following:

```python
###########################################################################
# ------------- Basic Settings --------------------------------------------
# ------------- MODIFY this block of code ---------------------------------
###########################################################################
VERSION = 'MyCoolExperiment' # this will be the name of the folder that gets created under 
                             # Training/output directory for your checkpoint & event files

NET_NAME = 'resnet152_v1d'  # must match one of the network names available in 
                            # the Training/data/pretrained_weights directory. 
                            # Currentlly suppoted networks are resnet_v1_50,
                            # resnet101_v1d, and resnet152_v1d. 

DATASET_NAME = 'OMITTED'    # The name you want to give your dataset. 

ROOT_PATH = '/home/Nightingale/Training' # The ROOT_PATH is the absolute path to the "Nightingale/Training" folder

TFRECORD_PATH = os.path.join(ROOT_PATH,'data/tfrecord_omitted') # (see Notebook #2) the directory that contains 
                                                               # (or will contain) the tensorflow record file. 
TFRECORD_NAME = DATASET_NAME+'_train.tfrecord' # # (see Notebook #2) the name of the Tensorflow Record composed of your training data

CLASS_NUM = 3 # number of classes in your dataset, not including background. 

GPU_GROUP = "0,1,2,3" # a list of GPU's to use for training

SAVE_WEIGHTS_INTE = 20000 # the frequency at which to write tensorflow checkpoint files

TRAIN_IMG_CHIP_SIZE = 2000 # Recommended pixel width/height setting for chiping NITF imagery for training. This 
                           # is approximately the max image size that can be processed during training with 
                           # resnet152_v1d on Tesla v100 SXM2 GPU's with ~34Gb of memory. 

TRAIN_IMG_CHIP_OVRLP = 200 # NITF image chip overlap for training. Consider increasing if image resolution or 
                           # pixels/object is larger than PRIORI objects. 
############################################################################
# -------------------- END Basic Settings ----------------------------------
# -------------------- END Modifications -----------------------------------
############################################################################
```

# <span style='color:Blue'> Step 0.2 - Edit the label dictionary to include your dataset categories and NMS threshold setting for training
    
* ## Using a text editor of your choice, open *label_dict.py* in Nightingale/ Training/ libs/ label_name_dict/ and edit the NAME_LABEL_MAP code block.

* ## Add your DATASET_NAME and object categories as a dictionary to the NAME_LABEL_MAP code block. Note that the DATASET_NAME must match the DATASET_NAME you put in the configuration (cfgs.py) file in the previous step. 

* ## Note that the NAME_LABEL_MAP will always include the 'back_ground' key with value 0.

* ## Also in the NAME_LABEL_MAP code block, add your training-NMS threshold dictionary. You can have a different setting for each category.

* ## You can overwrite the exisiting code block or add to it with an `elif` statement for a new dataset. For example, if your dataset is different types of aircraft, you can add to the code block with:

``` python
####################################
##-------NAME_LABEL_MAP block------#
####################################
if cfgs.DATASET_NAME == 'PRIORI3':
    NAME_LABEL_MAP = {
        'back_ground': 0,
        'class1': 1,
        'class2': 2,
        'class3': 3
    }
    
    # Set the training NMS threshold for each class
    threshold = {'class1':0.3,'class2':0.3,'class3':0.3} # these are hyperparameters you can play with

elif cfgs.DATASET_NAME == 'AIRCRAFTS':
    NAME_LABEL_MAP = {
        'back_ground': 0,
        'airplane': 1,
        'hot-air-balloon': 2,
        'blimp': 3,
        'glider': 4,
        'paramotor': 5,
        'helicopter': 6
    }
    
    # Set the training NMS threshold for each class
    threshold = {'airplane':0.3,'hot-air-balloon':0.3,'blimp':0.3,'glider':0.3,'paramotor':0.3,'helicopter' :0.3} # these are hyperparameters you can play with
    
else:
    assert 'please set label dict!'
####################################
##----END NAME_LABEL_MAP block-----#
####################################
```

<div class = "alert alert-block alert-success">

# <span style='color:Blue'> Once you've finished your edits to the config file and label dictionary, move on to NOTEBOOK-1 for pre-processing